<a href="https://colab.research.google.com/github/Vakhranev/Heritage/blob/main/%D0%98%D1%81%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BF%D0%B0%D1%80%D1%81%D0%B5%D1%80%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import json
import re
from collections import Counter

# Функция для очистки текста от знаков препинания и приведения к нижнему регистру
def clean_text(text):
    # Приводим текст к нижнему регистру
    text = text.lower()
    # Удаляем знаки препинания, кроме дефисов внутри слов
    return re.sub(r'(?<!\w)-|[^\w\s-]', '', text)

# Функция для подсчёта биграмм в тексте
def count_bigrams(text):
    tokens = text.split()
    bigrams = zip(tokens, tokens[1:])
    bigram_freq = Counter(bigrams)
    return bigram_freq

# Загрузка значений t_score из файла
def load_t_scores(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError: {e}")
        print("Попробуем открыть файл в другой кодировке...")
        with open(filename, 'r', encoding='latin1') as f:
            return json.load(f)

# Функция для проверки и корректировки биграммы с ё
def check_yo_in_bigram(bigram, t_scores):
    bigram_yo = bigram.replace('ё', 'е')
    t_score = t_scores.get(bigram, 0)

    # Если t_score == 0, проверяем биграмму с "е"
    if t_score == 0 and bigram_yo != bigram:
        t_score_yo = t_scores.get(bigram_yo, 0)
        if t_score_yo != 0:
            return t_score_yo  # Возвращаем t_score для биграммы с "е"

    return t_score  # Возвращаем исходный t_score

def process_america_texts(input_json, t_score_file, output_file):
    # Загрузка текстов
    with open(input_json, 'r', encoding='utf-8') as f:
        america_texts = json.load(f)

    # Загрузка t_scores
    t_scores = load_t_scores(t_score_file)

    result = {}

    for key, data in america_texts.items():
        # Исходный текст
        text = data["text"]

        # Очистка текста
        clean_text_content = clean_text(text)

        # Токенизация текста
        tokens = clean_text_content.split()

        # Подсчёт частоты биграмм
        bigram_freq = count_bigrams(clean_text_content)
        bigram_freq = {" ".join(bigram): freq for bigram, freq in bigram_freq.items()}

        # Формирование биграмм с t_score
        bigrams_unique = {}
        for bigram, freq in bigram_freq.items():
            t_score = check_yo_in_bigram(bigram, t_scores)  # Используем t_score с проверкой ё/е
            bigrams_unique[bigram] = {"t_score": t_score}

        # Сохранение результатов
        result[key] = {
            "text": text,
            "tokens": tokens,
            "bigrams_freq": bigram_freq,
            "bigrams_unique": bigrams_unique,
        }

    # Запись в файл
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

# Пример использования
input_json = "america_texts.json"
t_score_file = "cleaned_t_score_file.json"
output_file = "updated_america_texts.json"

process_america_texts(input_json, t_score_file, output_file)